<DIV ALIGN=CENTER>

# Introduction to Map/Reduce
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

In this IPython Notebook, we 

-----




-----
### Mapper: Word Count

The first Python code we will write is the map Python program. This
program simply reads data from STDIN, tokenizes each line into words and
outputs each word on a separate line along with a count of one. Thus our
map program generates a list of word tokens as the keys and the value is
always one.

-----

In [1]:
%%writefile /home/data_scientist/hadoop/mapper.py
#!/usr/bin/env python3

import sys

# We explicitly define the word/count separator token.
sep = '\t'

# We open STDIN and STDOUT
with sys.stdin as fin:
    with sys.stdout as fout:
    
        # For every line in STDIN
        for line in fin:
        
            # Strip off leading and trailing whitespace
            line = line.strip()
            
            # We split the line into word tokens. Use whitespace to split.
            # Note we don't deal with punctuation.
            
            words = line.split()
            
            # Now loop through all words in the line and output

            for word in words:
                fout.write("{0}{1}1\n".format(word, sep))

Overwriting /home/data_scientist/hadoop/mapper.py


-----

### Reducer: Word Count

The second Python program we write is our reduce program. In this code,
we read key-value pairs from STDIN and use the fact that the Hadoop
process first sorts all key-value pairs before sending the map output to
the reduce process to accumulate the cumulative count of each word. The
following code could easily be made more sophisticated by using `yield`
statements and iterators, but for clarity we use the simple approach of
tracking when the current word becomes different than the previous word
to output the key-cumulative count pairs.

-----

In [2]:
%%writefile /home/data_scientist/hadoop/reducer.py
#!/usr/bin/env python3

import sys

# We explicitly define the word/count separator token.
sep = '\t'

# We open STDIN and STDOUT
with sys.stdin as fin:
    with sys.stdout as fout:
    
        # Keep track of current word and count
        cword = None
        ccount = 0
        word = None
   
        # For every line in STDIN
        for line in fin:
        
            # Strip off leading and trailing whitespace
            # Note by construction, we should have no leading white space
            line = line.strip()
            
            # We split the line into a word and count, based on predefined
            # separator token.
            # Note we haven't dealt with punctuation.
            
            word, scount = line.split('\t', 1)
            
            # We wil assume count is always an integer value
            
            count = int(scount)
            
            # word is either repeated or new
            
            if cword == word:
                ccount += count
            else:
                # We have to handle first word explicitly
                if cword != None:
                    fout.write("{0:s}{1:s}{2:d}\n".format(cword, sep, ccount))
                
                # New word, so reset variables
                cword = word
                ccount = count
        else:
            # Output final word count
            if cword == word:
                fout.write("{0:s}{1:s}{2:d}\n".format(word, sep, ccount))

Overwriting /home/data_scientist/hadoop/reducer.py


-----
### Testing Python Map-Reduce

Before we begin using Hadoop, we should first test our Python codes out
to ensure they work as expected. First, we should change the permissions
of the two programs to be executable, which we can do with the Unix
`chmod` command.

-----

In [3]:
%%bash

chmod u+x /home/data_scientist/hadoop/mapper.py
chmod u+x /home/data_scientist/hadoop/reducer.py

ls -la /home/data_scientist/hadoop

total 1556
drwxr-xr-x  2 data_scientist users    4096 Apr  7 20:03 .
drwxr-xr-x 18 data_scientist users    4096 Apr  7 18:17 ..
-rw-r--r--  1 data_scientist users 1573151 Apr  7 18:17 book.txt
-rwxr--r--  1 data_scientist users     694 Apr  7 20:07 mapper.py
-rwxr--r--  1 data_scientist users    1481 Apr  7 20:07 reducer.py


-----

#### Testing Mapper.py

To test out the map Python code, we can run the Python `mapper.py` code
and specify that the code should redirect STDIN to read the book text
data. This is done in the following code cell, we pipe the output into
the Unix `head` command in order to restrict the output, which would be
one line per word found in the book text file. In the second code cell,
we next pipe the output of  `mapper.py` into the Unix `sort` command,
which is done automatically by Hadoop. To see the result of this
operation, we next pipe the result into the Unix `uniq` command to count
duplicates, pipe this result into a new sort routine to sort the output
by the number of occurrences of a word, and finally display the last few
lines with the Unix `tail` command to verify the program is operating
correctly.

-----

In [4]:
%%bash

cd /home/data_scientist/hadoop

./mapper.py <  book.txt | wc -l

267976


In [5]:
%%bash

cd /home/data_scientist/hadoop

./mapper.py <  book.txt | sort -n -k 1 | \
 uniq -c -d | sort -n -k 1 | tail -10

   2391 with	1
   2432 I	1
   2712 he	1
   3035 his	1
   4606 in	1
   4787 to	1
   5842 a	1
   6542 and	1
   8127 of	1
  13600 the	1


-----

#### Testing Reducer.py

To test out the reduce Python code, we run the previous code cell, but
rather than piping the result into the Unix `tail` command, we pipe the
result of the sort command into the Python `reducer.py` code. This
simulates the Hadoop model, where the map output is key sorted before
being passed into the reduce process. First, we will simply count the
number of lines displayed by the reduce process, which will indicate the
number of  unique _word tokens_ in the book. Next, we will sort the
output by the number of times each word token appears and display the
last few lines to compare with the previous results.

-----

In [6]:
%%bash

cd /home/data_scientist/hadoop

./mapper.py <  book.txt | sort -n -k 1 | \
./reducer.py | wc -l

50106


In [7]:
%%bash

cd /home/data_scientist/hadoop

./mapper.py <  book.txt | sort -n -k 1 | \
./reducer.py | sort -n -k 2 | tail -10

with	2391
I	2432
he	2712
his	3035
in	4606
to	4787
a	5842
and	6542
of	8127
the	13600


### Python Map/Reduce

At this point, we first need to change into the directory where we
created our Python mapper and reducer programs, and where we downloaded
the hadoop-streaming jar file and the sample book to analyze. In the
Hadoop Docker container, enter `cd rppds/hadoop`, which will change our
current working directory to the appropriate location, which is
indicated by a change in the shell prompt to `/rppds/hadoop#`. 

Before proceeding, we should test our Python codes, but now within the
Hadoop Docker container, which will have a different python environment
than our class container. We can easily do this by modifying our earlier
test to now use the correct path in the Hadoop Docker container:

    /rppds/hadoop/mapper.py <  book.txt | sort -n -k 1 |  \
        /rppds/hadoop/reducer.py | sort -n -k 2 | tail -10

Doing this, however, now gives an `UnicodeDecodeError`. The simplest
solution is to explicitly state that the Python interpreter should use
`utf-8` for all IO operations, which we can do by setting the Python
environment variable `PYTHONIOENCODING` to `utf-8`. We do this by
entering the following command at the container prompt:

    export PYTHONIOENCODING=utf-8

After setting this environment variable, the previous Unix command
string will now produce the correct output.


-----




## Python Hadoop Streaming

We are now ready tio actually run our Python codes via Hadoop Streaming.
The main command to perform this task is `$HADOOP_PREFIX/bin/hadoop jar
hs.jar`, where `hs.jar` is the hadoop-streaming jar file we downloaded
earlier in this Notebook. Running this command will display a usage
message that is not extremely useful, supplying the `-help` flag will
provide more a more useful summary. For our map/reduce Python example to
run successfully, we will need to specify six flags:

1. `-files`: a comma separated list of files to be copied to the Hadoop cluster.
2. `-input`: the HDFS input file(s) to be used for the map task.
3. `-output`: the HDFS output directory, used for the reduce task.
4. `-mapper`: the command to run for the map task.
5. `-reducer`: the command to run for the reduce task.
6. `-cmdenv`: set environment variables for a Hadoop streaming task.

Given our previous setup, we will run the full command as follows:

    $HADOOP_PREFIX/bin/hadoop jar hs.jar -files mapper.py,reducer.py -input wc/in \
        -output wc/out -mapper mapper.py -reducer reducer.py -cmdenv PYTHONIOENCODING=utf-8

When this command is run, a series of messages will be displayed to the
screen (STDOUT) showing the progress of our Hadoop Streaming task. At
the end of the stream of information messages will be a statement
indicating the location of the output directory as shown below:

In [8]:
%%bash

# Change into correct working directory
cd $HOME/hadoop

# Delete output directory (if it exists)
$HADOOP_PREFIX/bin/hdfs dfs -rm -r -f wc/out

# Grab current streaming lib jar filename
streaming_file=$(ls $HADOOP_PREFIX/share/hadoop/tools/lib/hadoop-streaming*)

# Run the Map Reduce task within Hadoop
$HADOOP_PREFIX/bin/hadoop jar $streaming_file \
    -files mapper.py,reducer.py -input wc/in \
    -output wc/out -mapper mapper.py -reducer reducer.py 

Deleted wc/out
packageJobJar: [/tmp/hadoop-unjar1976082841072322793/] [] /tmp/streamjob5549391610319238472.jar tmpDir=null


16/04/07 20:08:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
16/04/07 20:08:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/04/07 20:08:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
16/04/07 20:08:06 INFO mapred.FileInputFormat: Total input paths to process : 1
16/04/07 20:08:06 INFO mapreduce.JobSubmitter: number of splits:2
16/04/07 20:08:06 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1459971273803_0021
16/04/07 20:08:07 INFO impl.YarnClientImpl: Submitted application application_1459971273803_0021
16/04/07 20:08:07 INFO mapreduce.Job: The url to track the job: http://e7d89fb87de4:8088/proxy/application_1459971273803_0021/
16/04/07 20:08:07 INFO mapreduce.Job: Running job: job_1459971273803_0021
16/04/07 20:08:15 INFO mapreduce.Job: Job job_1459971273803_0021 running in uber mode : false
16/04/07 20:08:15 INFO mapreduce.Job:  map 0% reduce 0%


In [9]:
!ls $HADOOP_PREFIX/share/hadoop/tools/lib/hadoop-streaming*

/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar


-----

### Hadoop Results

In order to view the results of our Hadoop Streaming task, we must use
HDFS DFS commands to examine the directory and files generated by our
Python Map/Reduce programs. The following list of DFS commands might
prove useful to view the results of this map/reduce job.

    $HADOOP_PREFIX/bin/hdfs dfs -ls wc

    $HADOOP_PREFIX/bin/hdfs dfs -ls wc/out

    $HADOOP_PREFIX/bin/hdfs dfs -count -h wc/out/part-00000

    $HADOOP_PREFIX/bin/hdfs dfs -tail wc/out/part-00000

-----

In [10]:
!$HADOOP_PREFIX/bin/hdfs dfs -ls wc/out

Found 2 items
-rw-r--r--   1 data_scientist supergroup          0 2016-04-07 20:08 wc/out/_SUCCESS
-rw-r--r--   1 data_scientist supergroup     527718 2016-04-07 20:08 wc/out/part-00000


In [11]:
!$HADOOP_PREFIX/bin/hdfs dfs -count -h wc/out/part-00000

           0            1            515.3 K wc/out/part-00000


In [12]:
!$HADOOP_PREFIX/bin/hdfs dfs -tail wc/out/part-00000 | tail -10

zoo.	1
zoological	1
zouave's	1
zrads,	2
zrads.	1
É	1
Élus,_	1
à	3
è	3
état_.	1


-----

To compare this map/reduce Hadoop Streaming task output to our previous
output, we can use the `$HADOOP_PREFIX/bin/hdfs dfs -cat
wc/out/part-00000 | sort -n -k 2 | tail -10`, which should be executed at
a Hadoop Docker container shell prompt. This code listing provides the
succesful output of this command, following a succesful map/reduce
processing task.

```
/rppds/hadoop# $HADOOP_PREFIX/bin/hdfs dfs -cat wc/out/part-00000 | \
    sort -n -k 2 | tail -10

with	2391
I	2432
he	2712
his	3035
in	4606
to	4787
a	5842
and	6542
of	8127
the	13600
```

-----

In [13]:
!$HADOOP_PREFIX/bin/hdfs dfs -cat wc/out/part-00000 | sort -n -k 2 | tail -10

with	2391
I	2432
he	2712
his	3035
in	4606
to	4787
a	5842
and	6542
of	8127
the	13600


### Hadoop Cleanup

Following the succesful run of our map/reduce Python programs, we have
created a new directory `wc/out`, which contains two files. If we wish
to rerun this Hadoop Streaming map/reduce task, we must either specify a
different output directory, or else we must clean up the results of the
previous run. To remove the output directory, we can simply use the DFS
`-rm -r -f -skipTrash wc/out` command, which will immediately delete the
`wc/out` directory. The successful completion of this command is
indicated by Hadoop, and this can also be verified by listing the
contents of the `wc` directory as shown in the following screenshot.


-----

In [14]:
# !$HADOOP_PREFIX/bin/hdfs dfs -r -f wc/out

-----

### Student Activity

In the preceding cells, we introduced general liner models by using
pymc3. Now that you have run the Notebook, go back and make the
following changes to see how the results change.

1. Change the number of sample points both up and down, how does the glm
fit change?
2. Replace the existing model by a higher order model (include third and
possibly higher order terms). How well does the corresponding GLM fit
the data?
3. One can use the Bayes factor to compare model fits (simply the ratio
of the posteriors of the two models). Compute the Bayes factor for the
linear and quadratic model fits to the original data.

-----